# Lab Objectives

This lab aims to show how to use Gaussian distribution to do anomaly detection, and false alarm management in anomaly detection.

Make sure the file 'gaussian_anomaly_detection.py', dataset 'network-logs.csv' and image 'Normal_Distribution_pdf.png' and this notebook are located in the same folder.

Please try out the following cells and run the python code in your notebook. 

***
***This is not an assignment and you do not need to submit it***

# Gaussian distribution

As a quick recap, let us review some characteristics of the Gaussian distribution. In mathematical terms, the Gaussian distribution (also known as the normal distribution) represents a probability distribution of random variables, which takes the following mathematical form:

$$
f(x) = \frac{1}{\sqrt{2\pi\sigma^2}} e^{-\frac{(x-\mu)^2}{2\sigma^2}}
$$

Here, $\mu$ represents the average, and $\sigma^2$ is the variance (which is representative of the variability of the data around the average value). In its standard form, the mean $\mu$ assumes the value of 0, and $\sigma^2$ assumes the value of 1.

The strength of the Gaussian distribution is the central limit theorem, which, in general terms, establishes that the average of the observational data of a random variable—extracted independently—converges to the normal value as the number of observations increases.

In other words, the observations, as their number increases, are distributed symmetrically (and with greater probability) around the mean $\mu$. 

<img src="Normal_Distribution_pdf.png" style="width:400px;height:250px;">

While they deviate from the average value (tending to be distributed in the left and right extremities) for increasing values of $\sigma$, the normal distribution is therefore adequately represented by the values assumed by $\mu$ and $\sigma$.

Similarly, it is possible to determine the probability with which the observations are distributed around the average value, in proportion to the value of the variance; in other words, we can determine the following:

- 68% of observations fall within the range between $\mu - \sigma$ and $\mu + \sigma$.
- 95% of observations fall within the range between $\mu - 2\sigma$ and $\mu + 2\sigma$.
- 99.7% of observations fall within the range between $\mu - 3\sigma$ and $\mu + 3\sigma$.

The empirical rule, also referred to as the three-sigma rule or 68-95-99.7 rule, is a statistical rule which states that for a normal distribution, almost all observed data will fall within three standard deviations of the mean or average.

# Anomaly detection using the Gaussian distribution

The Gaussian distribution can be used to identify the outliers. The anomaly element consists of a significant difference assumed by the outliers with respect to the rest of the data. The more the majority of the data is firmly concentrated around the mean value $\mu$ with a low variance $\sigma$, the more significant the anomalous value assumed by the outlier is.

To use the Gaussian distribution in anomaly detection, we will perform the following steps:

1. Assume that the features of the training set are normally distributed (this can also be verified intuitively from the visual analysis of plotted data)
2. Estimate the $\mu$ and $\sigma$ values, representative of the distribution
3. Choose an adequate threshold, representative of the probability that the observations are anomalous
4. Assess the reliability of the algorithm

In the following example, we will show an implementation of Gaussian anomaly detection.

First of all, let's import the necessary Python libraries, and then load the data from a .csv file that represents the latency and network throughput values of each data stream we detected:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

dataset = pd.read_csv('network-logs.csv')

Once the data is loaded into memory, we can verify whether the distribution of the samples might resemble a Gaussian distribution, displaying the corresponding values in the form of a histogram:

In [ ]:
hist_dist = dataset[['LATENCY', 'THROUGHPUT']].hist(grid=False, figsize=(10,4))

At this point, we perform the data plotting on a scatter diagram, visually identifying the possible outliers:

In [ ]:
data = dataset[['LATENCY', 'THROUGHPUT']].values

plt.scatter(data[:, 0], data[:, 1], alpha=0.6)
plt.xlabel('LATENCY')
plt.ylabel('THROUGHPUT')
plt.title('DATA FLOW')
plt.show()

As can also be seen visually, most of the observations are concentrated around the average values, except for some cases. We therefore want to verify whether the anomalous cases are real, and then we proceed to estimate the representative values, $\mu$ and $\sigma$, of the underlying Gaussian distribution:

In [ ]:
'''
The anomaly detection module is implemented in the python file 'gaussian_anomaly_detection.py'
Make sure that both the 'gaussian_anomaly_detection.py' file and this notebook are located in the same folder.
'''
from gaussian_anomaly_detection import GaussianAnomalyDetection

gaussian_anomaly_detection = GaussianAnomalyDetection(data)

print('mu param estimation: ')
print(gaussian_anomaly_detection.mu_param)

print('\n')

print('sigma squared estimation: ')
print(gaussian_anomaly_detection.sigma_squared)

We then proceed to estimate the probabilities and threshold value, which we can then compare to identify the anomalous data:

In [ ]:
targets = dataset['ANOMALY'].values.reshape((data.shape[0], 1))
probs = gaussian_anomaly_detection.gaussian(data)

(threshold, F1, precision_, recall_, f1_) = gaussian_anomaly_detection.select_threshold(targets, probs)

print('\n')

print('threshold estimation: ')
print(threshold)

At this point, we are able to identify the outliers by comparing the individual probabilities of the samples with the previously estimated optimal threshold value, visualizing their presence in a scatter diagram:

In [ ]:
outliers = np.where(probs < threshold)[0]

# Plot original data.
plt.scatter(data[:, 0], data[:, 1], alpha=0.6, label='Dataset')
plt.xlabel('LATENCY')
plt.ylabel('THROUGHPUT')
plt.title('DATA FLOW')

# Plot the outliers.
plt.scatter(data[outliers, 0], data[outliers, 1], alpha=0.6, c='red', label='Outliers')

# Display plots.
plt.legend()
plt.plot()
plt.show()

# False alarm management in anomaly detection

Next we will make some assessments on the estimates made by the algorithm. In the case of IDS based on signatures, the risk of error is represented by the high number of false negatives, that is, attacks that are not detected. It is the same type of risk that we incur when using antivirus software. When a correspondence with the suspicious signature is not found, the IDS does not detect any anomalies.

On the other hand, in the case of anomaly-driven IDS, which is programmed to detect anomalies automatically, we face the risk of having a high number of false positives; that is, anomalies that are detected despite not being harmful.

To adequately manage these false alarms, we can use F1 score to estimate these errors. 
F1 can be used to evaluate the results obtained from Gaussian anomaly detection. The best estimates are obtained with F1 values close to 1, while the worst estimates correspond to F1 values that are close to 0.

In our example of Gaussian anomaly detection, the value of F1 is as follows:

In [ ]:
print('F1 score: ')
print(F1)

This F1 value is rather close to 1, which does not surprise us, since in choosing the best threshold, our Gaussian anomaly detection model selects the value that corresponds to the highest F1 score.

References

- Oleksii Trekhleb: https://github.com/trekhleb/homemade-machine-learning/blob/master/homemade/anomaly_detection/gaussian_anomaly_detection.py

- Hands On AI for Cybersecurity - Anomaly detection 


---
***end***